# Projet 8 - Déployez un modèle dans le cloud

## Sommaire

In [0]:
! pip3 freeze

In [0]:
import awscli
import pandas as pd
from PIL import Image
import numpy as np
import io
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

In [0]:
! aws s3 ls s3://databricks-workspace-stack-9d948-bucket/data/

In [0]:
! aws s3 cp s3://databricks-workspace-stack-9d948-bucket/data/Test1/Apricot/3_100.jpg .

In [0]:
PATH = 's3://databricks-workspace-stack-9d948-bucket/'
PATH_Data = PATH+'data/Test1/'
PATH_Result = PATH+'data/Results/'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

In [0]:
dbutils.fs.ls("s3://databricks-workspace-stack-9d948-bucket/data/Test1/Apricot/")

## Création de la SparkSession

L’application Spark est contrôlée grâce à un processus de pilotage (driver process) appelé **SparkSession**. <br />
<u>Une instance de **SparkSession** est la façon dont Spark exécute les fonctions définies par l’utilisateur <br />
dans l’ensemble du cluster</u>. <u>Une SparkSession correspond toujours à une application Spark</u>.

<u>Ici nous créons une session spark en spécifiant dans l'ordre</u> :
 1. un **nom pour l'application**, qui sera affichée dans l'interface utilisateur Web Spark "**P8**"
 2. que l'application doit s'exécuter **localement**. <br />
   Nous ne définissons pas le nombre de cœurs à utiliser (comme .master('local[4]) pour 4 cœurs à utiliser), <br />
   nous utiliserons donc tous les cœurs disponibles dans notre processeur.<br />
 3. une option de configuration supplémentaire permettant d'utiliser le **format "parquet"** <br />
   que nous utiliserons pour enregistrer et charger le résultat de notre travail.
 4. vouloir **obtenir une session spark** existante ou si aucune n'existe, en créer une nouvelle

In [0]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [0]:
sc = spark.sparkContext

<u>Affichage des informations de Spark en cours d'execution</u> :

In [0]:
spark

## Traitement des données

<u>Dans la suite de notre flux de travail, <br />
nous allons successivement</u> :
1. Préparer nos données
    1. Importer les images dans un dataframe **pandas UDF**
    2. Associer aux images leur **label**
    3. Préprocesser en **redimensionnant nos images pour <br />
       qu'elles soient compatibles avec notre modèle**
2. Préparer notre modèle
    1. Importer le modèle **MobileNetV2**
    2. Créer un **nouveau modèle** dépourvu de la dernière couche de MobileNetV2
3. Définir le processus de chargement des images et l'application <br />
   de leur featurisation à travers l'utilisation de pandas UDF
3. Exécuter les actions d'extraction de features
4. Enregistrer le résultat de nos actions
5. Tester le bon fonctionnement en chargeant les données enregistrées




### Chargement des données

Les images sont chargées au format binaire, ce qui offre, <br />
plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger <br />
uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus <br />
dans les sous-dossiers du dossier communiqué.

In [0]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [0]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

### Préparation du modèle

Je vais utiliser la technique du **transfert learning** pour extraire les features des images.<br />
J'ai choisi d'utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée <br />
à d'autres modèles comme *VGG16* par exemple.

Pour en savoir plus sur la conception et le fonctionnement de MobileNetV2, <br />
je vous invite à lire [cet article](https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c).

Il existe une dernière couche qui sert à classer les images <br />
selon 1000 catégories que nous ne voulons pas utiliser.<br />
L'idée dans ce projet est de récupérer le **vecteur de caractéristiques <br />
de dimensions (1,1,1280)** qui servira, plus tard, au travers d'un moteur <br />
de classification à reconnaitre les différents fruits du jeu de données.

Comme d'autres modèles similaires, **MobileNetV2**, lorsqu'on l'utilise <br />
en incluant toutes ses couches, attend obligatoirement des images <br />
de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), <br />
nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [0]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [0]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [0]:
new_model.summary()

Tous les workeurs doivent pouvoir accéder au modèle ainsi qu'à ses poids. <br />
Une bonne pratique consiste à charger le modèle sur le driver puis à diffuser <br />
ensuite les poids aux différents workeurs.

In [0]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [0]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [0]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

### 3.7.4 Exécution des actions d'extraction de features

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), <br />
peuvent rencontrer des erreurs de type Out Of Memory (OOM).<br />
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, <br />
essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Je n'utiliserai pas cette commande dans ce projet <br />
et je laisse donc la commande en commentaire.

In [0]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

Nous pouvons maintenant exécuter la featurisation sur l'ensemble de notre DataFrame Spark.<br />
<u>REMARQUE</u> : Cela peut prendre beaucoup de temps, tout dépend du volume de données à traiter. <br />

Notre jeu de données de **Test** contient **22819 images**. <br />
Cependant, dans l'exécution en mode **local**, <br />
nous <u>traiterons un ensemble réduit de **330 images**</u>.

In [0]:
features_df = images.repartition(10).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

<u>Rappel du PATH où seront inscrits les fichiers au format "**parquet**" <br />
contenant nos résultats, à savoir, un DataFrame contenant 3 colonnes</u> :
 1. Path des images
 2. Label de l'image
 3. Vecteur de caractéristiques de l'image

In [0]:
print(PATH_Result)

<u>Enregistrement des données traitées au format "**parquet**"</u> :

In [0]:
features_df.write.mode("overwrite").parquet(PATH_Result)

## Chargement des données enregistrées et validation du résultat

<u>On charge les données fraichement enregistrées dans un **DataFrame Pandas**</u> :

In [0]:
df = spark.read.parquet(PATH_Result)

<u>On affiche les 5 premières lignes du DataFrame</u> :

In [0]:
df = df.toPandas()
df.head()

<u>On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280</u> :

In [0]:
df.loc[0,'features'].shape

In [0]:
df.shape

Nous venons de valider le processus sur un jeu de données allégé en local <br />
où nous avons simulé un cluster de machines en répartissant la charge de travail <br />
sur différents cœurs de processeur au sein d'une même machine.

Nous allons maintenant généraliser le processus en déployant notre solution <br />
sur un réel cluster de machines et nous travaillerons désormais sur la totalité <br />
des 22819 images de notre dossier "Test".

## Analyse en composantes principales 

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType, DoubleType, StringType

In [0]:
df.columns

In [0]:
df['features'][0]

In [0]:
features_df.printSchema()

In [0]:
features_df.columns

In [0]:
features_df.show()

In [0]:
features_df.first()

In [0]:
from pyspark.ml.feature import PCA
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a UDF to convert the array<float> to VectorUDT
array_to_vector_udf = udf(lambda arr: Vectors.dense(arr), VectorUDT())
# Assuming 'features_df' is your PySpark DataFrame
# Apply the UDF to create a new column of type VectorUDT
features_df = features_df.withColumn("features_vector", array_to_vector_udf(features_df["features"]))

In [0]:
features_df.show()

In [0]:
# Assuming 'features_df' is your PySpark DataFrame
num_components = 128 
pca = PCA(k=num_components, inputCol='features_vector', outputCol='pca_features')
pca_model = pca.fit(features_df)
result_df = pca_model.transform(features_df)

In [0]:
result_df.show()

In [0]:
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql.functions import col

# Assuming 'result_df' contains the PCA-transformed features and labels
# Calculate the explained variance ratio for each principal component
# Assuming 'explained_variance' contains the explained variance for each principal component
explained_variance = pca_model.explainedVariance.toArray()  # Convert to a NumPy array
scree = (explained_variance*100).round(2) 
# Calculate the cumulative explained variance
scree_cum = np.cumsum(scree)

x_list = range(1, len(scree[:10])+1)
list(x_list)
sns.set()
fig = plt.bar(x_list, scree[:10])
plt.plot(x_list, scree_cum[:10],c="red",marker='o')
plt.xlabel("Rang de l'axe d'inertie")
plt.ylabel("Pourcentage d'inertie")
plt.title("Éboulis des valeurs propres")

# Show the combined plot
plt.show()
#plt.savefig('s3://oc-calculsdistribues-p8-jkm-rob/data/Results/eboulis_valeurs.png')

In [0]:
explained_variance

In [0]:
pandas_df = result_df.toPandas()

In [0]:
pandas_df.head()

In [0]:
# Extract the PCA components (the first two)
for j in range(6):
    feat_name = 'pca'+str(j+1)
    pandas_df[feat_name] = pandas_df['pca_features'].apply(lambda x: x[j])

In [0]:
sns.set()
fig, ax = plt.subplots(figsize= (8,8))
sns.scatterplot(data = pandas_df, x = 'pca1', y = 'pca2', hue = 'label')
plt.savefig('PCA_plan_principale_1.png')


In [0]:
sns.set()
fig, ax = plt.subplots(figsize= (8,8))
sns.scatterplot(data = pandas_df, x = 'pca3', y = 'pca4', hue = 'label')
plt.savefig('PCA_plan_principale_2.png')

In [0]:
pandas_df = pandas_df.drop(columns =['features', 'features_vector', 'pca_features'])
pandas_df.head()

In [0]:
pandas_df.to_csv('KuohMoukouri_Joyce_4_matrice_092023.csv')

In [0]:
! ls

In [0]:
! aws s3 cp KuohMoukouri_Joyce_4_matrice_092023.csv s3://databricks-workspace-stack-9d948-bucket/